# Exploratory Data Analysis - Stock Price Prediction
## 34-Year Historical Market Data (1990-2024)

This notebook performs comprehensive data exploration including:
- Data quality assessment
- Temporal analysis and trend visualization
- Statistical profiling
- Feature relationship analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

## 1. Data Loading and Initial Inspection

In [ ]:
# Load dataset
df = pd.read_csv('../stock_data.csv', parse_dates=['dt'])
df.set_index('dt', inplace=True)

print(f"Dataset Shape: {df.shape}")
print(f"\nDate Range: {df.index.min()} to {df.index.max()}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Data types and memory usage
df.info()

## 2. Data Quality Assessment

In [ ]:
# Missing values analysis
missing_data = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100
}).sort_values('Missing_Count', ascending=False)

print("Missing Values Summary:")
print(missing_data[missing_data['Missing_Count'] > 0])

# Visualize missing data
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
plt.title('Missing Data Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
# Check for date continuity and gaps
date_diff = df.index.to_series().diff()
gaps = date_diff[date_diff > pd.Timedelta(days=3)]  # Gaps > 3 days (excluding weekends)

print(f"\nNumber of significant gaps (>3 days): {len(gaps)}")
if len(gaps) > 0:
    print("\nTop 10 largest gaps:")
    print(gaps.nlargest(10))

In [ ]:
# Detect negative or impossible values
print("Data Range Validation:")
print(f"\nNegative prices in sp500: {(df['sp500'] < 0).sum()}")
print(f"Negative volumes in sp500_volume: {(df['sp500_volume'] < 0).sum()}")
print(f"Negative VIX: {(df['vix'] < 0).sum()}")

# Basic statistics
print("\nBasic Statistics:")
df.describe()

## 3. Temporal Analysis and Visualization

In [ ]:
# Plot S&P 500 historical trends with major events
fig, ax = plt.subplots(figsize=(16, 8))
ax.plot(df.index, df['sp500'], linewidth=1.5, color='steelblue', label='S&P 500')

# Annotate major market events
events = [
    ('2000-03-10', 'Dot-com Crash', 'red'),
    ('2008-09-15', '2008 Financial Crisis', 'red'),
    ('2020-03-11', 'COVID-19 Pandemic', 'red'),
    ('2009-03-09', 'Market Bottom 2009', 'green'),
    ('2020-03-23', 'COVID Bottom', 'green')
]

for date, label, color in events:
    if pd.Timestamp(date) in df.index:
        ax.axvline(pd.Timestamp(date), color=color, linestyle='--', alpha=0.7, linewidth=1)
        ax.text(pd.Timestamp(date), ax.get_ylim()[1]*0.95, label, 
                rotation=90, verticalalignment='top', fontsize=9)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('S&P 500 Price', fontsize=12)
ax.set_title('S&P 500 Historical Price (1990-2024) with Major Market Events', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Stationarity test (Augmented Dickey-Fuller)
def adf_test(series, name):
    result = adfuller(series.dropna())
    print(f"\nADF Test for {name}:")
    print(f"  ADF Statistic: {result[0]:.6f}")
    print(f"  p-value: {result[1]:.6f}")
    print(f"  Critical Values:")
    for key, value in result[4].items():
        print(f"    {key}: {value:.3f}")
    if result[1] <= 0.05:
        print(f"  Result: STATIONARY (reject H0)")
    else:
        print(f"  Result: NON-STATIONARY (fail to reject H0)")

adf_test(df['sp500'], 'S&P 500')
adf_test(df['vix'], 'VIX')

In [ ]:
# Calculate returns for stationarity
df['sp500_returns'] = df['sp500'].pct_change()
adf_test(df['sp500_returns'].dropna(), 'S&P 500 Returns')

In [ ]:
# Volatility clustering visualization
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# S&P 500 Returns
axes[0].plot(df.index, df['sp500_returns'], linewidth=0.5, color='navy', alpha=0.7)
axes[0].set_title('S&P 500 Daily Returns - Volatility Clustering', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Returns')
axes[0].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[0].grid(True, alpha=0.3)

# VIX
axes[1].plot(df.index, df['vix'], linewidth=1, color='darkred', alpha=0.8)
axes[1].set_title('VIX (Fear Index) Over Time', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('VIX')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Statistical Profiling

In [ ]:
# Descriptive statistics with skewness and kurtosis
stats_df = df.describe().T
stats_df['skewness'] = df.skew()
stats_df['kurtosis'] = df.kurtosis()

print("Extended Descriptive Statistics:")
stats_df

In [ ]:
# Distribution plots for key features
features_to_plot = ['sp500', 'vix', 'sp500_volume', 'us3m', 'epu', 'GPRD']

fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, col in enumerate(features_to_plot):
    axes[idx].hist(df[col].dropna(), bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    axes[idx].set_title(f'Distribution of {col}', fontweight='bold')
    axes[idx].set_xlabel(col)
    axes[idx].set_ylabel('Frequency')
    axes[idx].grid(True, alpha=0.3)
    
    # Add normal distribution overlay
    mu, sigma = df[col].mean(), df[col].std()
    x = np.linspace(df[col].min(), df[col].max(), 100)
    axes[idx].plot(x, stats.norm.pdf(x, mu, sigma) * len(df[col].dropna()) * (df[col].max() - df[col].min()) / 50, 
                   'r-', linewidth=2, label='Normal Dist')
    axes[idx].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
correlation_matrix = df.corr()

# Heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1)
plt.title('Pearson Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Top correlations with S&P 500
sp500_corr = correlation_matrix['sp500'].sort_values(ascending=False)
print("\nTop Correlations with S&P 500:")
print(sp500_corr)

In [ ]:
# Spearman correlation (non-linear relationships)
spearman_corr = df.corr(method='spearman')

plt.figure(figsize=(12, 10))
sns.heatmap(spearman_corr, annot=True, fmt='.2f', cmap='viridis', 
            center=0, square=True, linewidths=1)
plt.title('Spearman Correlation Matrix (Rank-based)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Multicollinearity check using VIF
# Select numeric columns excluding target
features_for_vif = ['vix', 'sp500_volume', 'djia', 'djia_volume', 'hsi', 
                    'ads', 'us3m', 'joblessness', 'epu', 'GPRD', 'prev_day']

vif_data = pd.DataFrame()
vif_data['Feature'] = features_for_vif
vif_data['VIF'] = [variance_inflation_factor(df[features_for_vif].dropna().values, i) 
                   for i in range(len(features_for_vif))]
vif_data = vif_data.sort_values('VIF', ascending=False)

print("\nVariance Inflation Factor (VIF):")
print("Note: VIF > 10 indicates high multicollinearity")
print(vif_data)

# Visualize VIF
plt.figure(figsize=(10, 6))
plt.barh(vif_data['Feature'], vif_data['VIF'], color='coral')
plt.axvline(x=10, color='red', linestyle='--', label='VIF=10 threshold')
plt.xlabel('VIF Score', fontsize=12)
plt.title('Variance Inflation Factor by Feature', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.show()

## 5. Feature Relationship Analysis

In [ ]:
# VIX vs S&P 500 relationship (typically inverse)
plt.figure(figsize=(12, 6))
plt.scatter(df['sp500'], df['vix'], alpha=0.3, s=10, c=df.index.year, cmap='viridis')
plt.colorbar(label='Year')
plt.xlabel('S&P 500', fontsize=12)
plt.ylabel('VIX (Volatility Index)', fontsize=12)
plt.title('VIX vs S&P 500: Inverse Relationship Analysis', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Correlation between VIX and S&P 500: {df['vix'].corr(df['sp500']):.4f}")

In [ ]:
# Volume analysis on high volatility days
df['abs_returns'] = df['sp500_returns'].abs()
high_vol_days = df['abs_returns'] > df['abs_returns'].quantile(0.95)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Volume comparison
axes[0].boxplot([df.loc[~high_vol_days, 'sp500_volume'].dropna(), 
                 df.loc[high_vol_days, 'sp500_volume'].dropna()],
                labels=['Normal Days', 'High Volatility Days'])
axes[0].set_ylabel('Volume', fontsize=12)
axes[0].set_title('Volume on Normal vs High Volatility Days', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# VIX comparison
axes[1].boxplot([df.loc[~high_vol_days, 'vix'].dropna(), 
                 df.loc[high_vol_days, 'vix'].dropna()],
                labels=['Normal Days', 'High Volatility Days'])
axes[1].set_ylabel('VIX', fontsize=12)
axes[1].set_title('VIX on Normal vs High Volatility Days', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Macroeconomic indicators impact
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Unemployment vs S&P 500
axes[0, 0].scatter(df['joblessness'], df['sp500'], alpha=0.5, s=10)
axes[0, 0].set_xlabel('Joblessness Quartile')
axes[0, 0].set_ylabel('S&P 500')
axes[0, 0].set_title('Unemployment vs Market Performance')
axes[0, 0].grid(True, alpha=0.3)

# Interest rates vs S&P 500
axes[0, 1].scatter(df['us3m'], df['sp500'], alpha=0.5, s=10, c='green')
axes[0, 1].set_xlabel('US 3-Month Treasury Yield')
axes[0, 1].set_ylabel('S&P 500')
axes[0, 1].set_title('Interest Rates vs Market Performance')
axes[0, 1].grid(True, alpha=0.3)

# EPU vs S&P 500
axes[1, 0].scatter(df['epu'], df['sp500'], alpha=0.5, s=10, c='orange')
axes[1, 0].set_xlabel('Economic Policy Uncertainty')
axes[1, 0].set_ylabel('S&P 500')
axes[1, 0].set_title('Policy Uncertainty vs Market Performance')
axes[1, 0].grid(True, alpha=0.3)

# GPRD vs S&P 500
axes[1, 1].scatter(df['GPRD'], df['sp500'], alpha=0.5, s=10, c='red')
axes[1, 1].set_xlabel('Geopolitical Risk')
axes[1, 1].set_ylabel('S&P 500')
axes[1, 1].set_title('Geopolitical Risk vs Market Performance')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Cross-market correlation analysis
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# DJIA vs S&P 500
axes[0].scatter(df['djia'], df['sp500'], alpha=0.4, s=10)
axes[0].set_xlabel('DJIA', fontsize=12)
axes[0].set_ylabel('S&P 500', fontsize=12)
axes[0].set_title(f'DJIA vs S&P 500 (Corr: {df["djia"].corr(df["sp500"]):.4f})', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# HSI vs S&P 500
axes[1].scatter(df['hsi'], df['sp500'], alpha=0.4, s=10, c='green')
axes[1].set_xlabel('Hang Seng Index', fontsize=12)
axes[1].set_ylabel('S&P 500', fontsize=12)
axes[1].set_title(f'HSI vs S&P 500 (Corr: {df["hsi"].corr(df["sp500"]):.4f})', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Key Findings Summary

In [ ]:
print("="*80)
print("EDA KEY FINDINGS")
print("="*80)

print("\n1. DATA QUALITY:")
print(f"   - Total observations: {len(df):,}")
print(f"   - Date range: {df.index.min().strftime('%Y-%m-%d')} to {df.index.max().strftime('%Y-%m-%d')}")
print(f"   - Missing values: {df.isnull().sum().sum()} total")
print(f"   - No negative prices or volumes detected")

print("\n2. STATIONARITY:")
print(f"   - S&P 500 price series is NON-STATIONARY (expected for price data)")
print(f"   - S&P 500 returns are STATIONARY (suitable for modeling)")
print(f"   - Differencing/returns transformation recommended for models")

print("\n3. CORRELATIONS:")
print(f"   - VIX vs S&P 500: {df['vix'].corr(df['sp500']):.4f} (inverse relationship confirmed)")
print(f"   - DJIA vs S&P 500: {df['djia'].corr(df['sp500']):.4f} (very high correlation)")
print(f"   - HSI vs S&P 500: {df['hsi'].corr(df['sp500']):.4f} (moderate international correlation)")

print("\n4. MULTICOLLINEARITY:")
high_vif = vif_data[vif_data['VIF'] > 10]
if len(high_vif) > 0:
    print(f"   - {len(high_vif)} features have VIF > 10:")
    for _, row in high_vif.iterrows():
        print(f"     * {row['Feature']}: {row['VIF']:.2f}")
else:
    print(f"   - No severe multicollinearity detected (all VIF < 10)")

print("\n5. VOLATILITY:")
print(f"   - Volatility clustering observed in returns")
print(f"   - VIX spikes during major crises (2008, 2020)")
print(f"   - Higher volume on high volatility days confirmed")

print("\n6. RECOMMENDATIONS FOR MODELING:")
print(f"   - Use returns instead of raw prices for stationarity")
print(f"   - Consider feature selection to address multicollinearity")
print(f"   - Implement temporal train/test split (NO shuffling)")
print(f"   - Account for volatility clustering in model selection")
print(f"   - Crisis periods (2008, 2020) may need special handling")

print("\n" + "="*80)

In [ ]:
# Save cleaned data for next phase
df.to_csv('../data/processed/eda_cleaned.csv')
print("\nCleaned data saved to: data/processed/eda_cleaned.csv")